# *** EDA Sistema de recomendación***

In [2]:
import pandas as pd

In [10]:
df = pd.read_csv("./DataSet/movies_dataset_Para_EDA.csv")

In [11]:
df["id"].count()

45346

In [13]:
df.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'release_year', 'return'],
      dtype='object')

In [36]:

plReco = df[["title", 'popularity']][df['title'].str.contains("Robin")]
plReco = plReco.sort_values(by="popularity", ascending= False)  
plReco.head(5)    

,title,popularity
39633,Robinson Crusoe,8.969705
9081,Robinson Crusoe,8.739166
16760,Pooh's Grand Adventure: The Search for Christo...,5.839180
45341,Robin Hood,5.683753
29138,Batman vs. Robin,5.489420


In [42]:
tex = "robin hood"
tex = tex.title()
tex

'Robin Hood'

In [49]:
def get_recommendation(titulo: str):
    df = pd.read_csv("./DataSet/movies_dataset_Para_EDA.csv")
    plReco = df[["title", 'popularity']][df['title'].str.contains(f"{titulo.title()}")]
    plReco = plReco.sort_values(by="popularity", ascending= False)  
    return plReco.head(5)

In [50]:
get_recommendation("robin")

,title,popularity
39633,Robinson Crusoe,8.969705
9081,Robinson Crusoe,8.739166
16760,Pooh's Grand Adventure: The Search for Christo...,5.839180
45341,Robin Hood,5.683753
29138,Batman vs. Robin,5.489420
